In [ ]:
import os
import pandas as pd
import requests
from dotenv import load_dotenv
from urllib.parse import urlencode

# Load environment variables
load_dotenv()

# API key from .env file
API_KEY = os.getenv('DISTANCE_MATRIX_API_KEY')

# Load the geocoded data
walmart_df = pd.read_csv('walmart_sample_geocoded.csv')
zips_df = pd.read_csv('zips_sample_geocoded.csv')

# Function to construct the API URL
def construct_api_url(origins, destinations, api_key):
    params = {
        'origins': '|'.join(origins),
        'destinations': '|'.join(destinations),
        'key': api_key
    }
    return f"https://api.distancematrix.ai/maps/api/distancematrix/json?{urlencode(params)}"

# Function to make the API request
def make_api_request(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Function to process the API response and extract travel times
def process_response(response, origins, destinations):
    travel_times = []
    if response and response['status'] == 'OK':
        rows = response['rows']
        for i, row in enumerate(rows):
            elements = row['elements']
            for j, element in enumerate(elements):
                if element['status'] == 'OK':
                    origin = origins[i]
                    destination = destinations[j]
                    time = element['duration']['text']
                    travel_times.append((origin, destination, time))
                else:
                    print(f"Error: {element['status']} for origin {origins[i]} and destination {destinations[j]}")
    return travel_times

# Initialize a list to store the results
results = []

# Iterate over each state
for state in zips_df['state_name'].unique():
    state_walmarts = walmart_df[walmart_df['State'] == state]
    state_zips = zips_df[zips_df['state_name'] == state]
    
    # Construct origins and destinations
    origins = state_zips['latitude'].astype(str) + ',' + state_zips['longitude'].astype(str)
    destinations = state_walmarts['latitude'].astype(str) + ',' + state_walmarts['longitude'].astype(str)
    
    # Construct the API URL
    url = construct_api_url(origins.tolist(), destinations.tolist(), API_KEY)
    
    # Make the API request
    response = make_api_request(url)
    
    # Process the response
    travel_times = process_response(response, origins.tolist(), destinations.tolist())
    results.extend(travel_times)

# Convert results to DataFrame and save to CSV
results_df = pd.DataFrame(results, columns=['Origin', 'Destination', 'Travel Time'])
results_df.to_csv('travel_times.csv', index=False)